# Example of ingesting cryptocurrency data with a third party integration

[CryptoFeed](https://github.com/bmoscon/cryptofeed) is an open source library to get data from multiple cryptocurrency exchange datafeeds and store the data in a fast database. One of the supported databases to store the data is QuestDB.

In this example we will get trades for the symbols `ADA-USDT`, `BTC-USDT`, `ETH-BTC`, and `ETH-USDT` from [OKX](https://www.okx.com/) and will store them in QuestDB.

Since the CryptoFeed library takes care of everything, we only need to point it to the QuestDB TCP socket connection (which defaults to port 9009) and the library will take care of everything.

When you start the notebook, it will start capturing data continuously until you stop it or until you close this tab on your browser.

This dataset is the same that is powering the demo at `https://demo.questdb.io`, with the only difference that the demo machine is tracking 32 symbols, rather than only four. 

The script below processes each message and it stores it
on QuestDB, in a table named `trades`, with the schema below. If table does not exist, it will be automatically created on the first write.

```sql
CREATE TABLE 'trades' ( 
	symbol SYMBOL CAPACITY 256 CACHE,
	side SYMBOL CAPACITY 256 CACHE,
	price DOUBLE,
	amount DOUBLE,
	timestamp TIMESTAMP
) timestamp(timestamp) PARTITION BY DAY WAL;
```

To see the live data on your database, you can open a new tab on your browser and navigate to `http://localhost:9000`. You can then execute a simple query like `SELECT * FROM trades -10;` to see the latest 10 trades. Or you could execute a sligthly more sophisticated query like `select timestamp, symbol, side, sum(price * amount) from trades sample by 1m ` to get the totals for each symbol at 1 minute intervals.

For more realistic queries, please open in a new tab the [Examples-of-market-data-queries notebook](/notebooks/Examples-of-market-data-queries.ipynb), where you will find some queries adapted from the demo machine that should return results for your dataset.

If you want to see your live data on a real-time dashboard, please navigate in a new tab to [the demo dashboard](http://localhost:3000/d/live-trades-demo/live-trades-demo) powered by Grafana. The user is `admin` and password `quest`






In [1]:
#
# This product based on cryptofeed https://github.com/bmoscon/cryptofeed software developed by Bryant Moscon (http://www.bryantmoscon.com/) Copyright (C) 2017-2022 Bryant Moscon - bmoscon@gmail.com
#

import multiprocessing
import os
import argparse
from datetime import datetime


import ipywidgets as widgets
from IPython.display import display
import threading
import time
from cryptofeed import FeedHandler
from cryptofeed.backends.backend import BackendCallback
from cryptofeed.backends.socket import SocketCallback
from cryptofeed.defines import TRADES
from cryptofeed.exchanges import OKX

QUEST_HOST = 'questdb'
QUEST_PORT = 9009

class QuestCallback(SocketCallback):
    def __init__(self, host='host.docker.internal', port=9009, **kwargs):
        super().__init__(f"tcp://{host}", port=port, **kwargs)
        self.numeric_type = float
        self.none_to = None

    async def writer(self):
        while True:
            try:
                await self.connect()
            except Exception as e:
                print("Connection error:", e)
                break
            async with self.read_queue() as update:
                update = "\n".join(update) + "\n"
                try:
                    self.conn.write(update.encode())
                except Exception as e:
                    print("Write error:", e)
                    break  

class TradeQuest(QuestCallback, BackendCallback):
    default_key = 'trades'

    async def write(self, data):
        update = f'{self.key},symbol={data["symbol"]},side={data["side"]} price={data["price"]},amount={data["amount"]} {int(data["timestamp"] * 1_000_000_000)}'
        await self.queue.put(update)
        
def run_feed():
    # In a separate process, we don't need to worry about Jupyter's event loop.
    handler = FeedHandler()
    symbols = [
        'ADA-USDT',
        'BTC-USDT',       
        'ETH-BTC',
        'ETH-USDT',
    ]
    
    handler.add_feed(OKX(channels=[TRADES], symbols=symbols,
                           callbacks={TRADES: TradeQuest(host=QUEST_HOST, port=QUEST_PORT)}))
    handler.run()


# Start the feed handler in a separate process.
feed_process = multiprocessing.Process(target=run_feed)
feed_process.start()

print("The process will run in the background until the notebook is stopped. Status will update every second")

# Create a status indicator widget
status = widgets.Label("Feed is running...")
display(status)

# Start a background thread that periodically updates the status label
def update_status():
    while feed_process.is_alive():
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        status.value = f"Feed is running... Last check: {timestamp}"
        time.sleep(1)
    status.value = "Feed stopped."


threading.Thread(target=update_status, daemon=True).start()


The process will run in the background until the notebook is stopped. Status will update every second


Label(value='Feed is running...')